## Homework 6

The goal of this homework is to create a tree-based regression model for prediction apartment prices (column `'price'`).

In this homework we'll again use the New York City Airbnb Open Data dataset - the same one we used in homework 2 and 3.

You can take it from [Kaggle](https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data?select=AB_NYC_2019.csv)
or download from [here](https://raw.githubusercontent.com/alexeygrigorev/datasets/master/AB_NYC_2019.csv)
if you don't want to sign up to Kaggle.

Let's load the data:

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns


%matplotlib inline

In [ ]:
columns = [
    'neighbourhood_group', 'room_type', 'latitude', 'longitude',
    'minimum_nights', 'number_of_reviews','reviews_per_month',
    'calculated_host_listings_count', 'availability_365','price'
]

target = 'price'

df = pd.read_csv('nyc.csv', usecols=columns)
df.reviews_per_month = df.reviews_per_month.fillna(0)

* Apply the log tranform to `price`
* Do train/validation/test split with 60%/20%/20% distribution. 
* Use the `train_test_split` function and set the `random_state` parameter to 1

In [ ]:
df.head()

In [ ]:
df['price'] = np.log1p(df[target])

In [ ]:
target = df['price']
df.drop(columns= ['price'], inplace= True)

In [ ]:
df.head()

In [ ]:
from sklearn.model_selection import train_test_split

# Split data
x_train, x_test, y_train, y_test = train_test_split(df, target, test_size=0.2, random_state=1)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=1)

Now, use `DictVectorizer` to turn train and validation into matrices:

In [ ]:
print(len(x_train), len(y_train))
print(len(x_val), len(y_val))

In [ ]:
from sklearn.feature_extraction import DictVectorizer

dv = DictVectorizer(sparse=False)

train_dict = x_train.to_dict(orient= 'records')
train_dv = dv.fit_transform(train_dict)

val_dict = x_val.to_dict(orient= 'records')
val_dv = dv.transform(val_dict)

## Question 1

Let's train a decision tree regressor to predict the price variable. 

* Train a model with `max_depth=1`

In [ ]:
from sklearn.tree import DecisionTreeRegressor
reg = DecisionTreeRegressor(max_depth= 1)

Which feature is used for splitting the data?

* `room_type`
* `neighbourhood_group`
* `number_of_reviews`
* `reviews_per_month`

> room_type=Entire home/apt

In [ ]:
reg.fit(train_dv, y_train)

In [ ]:
imp_feature = np.argmax(reg.feature_importances_)

dv.get_feature_names()[imp_feature]

In [ ]:
from sklearn.tree import export_text
print(export_text(reg, feature_names=dv.get_feature_names()))

## Question 2

Train a random forest model with these parameters:

* `n_estimators=10`
* `random_state=1`
* `n_jobs=-1`  (optional - to make training faster)

>Validation RMSE: 0.211

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=10, random_state= 1, n_jobs= -1)

In [ ]:
rf.fit(train_dv, y_train)

In [ ]:
y_pred = rf.predict(val_dv)

In [ ]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_true= y_val, y_pred = y_pred, squared= False)
print(f'Validation RMSE: {round(rmse, 3)}')

What's the RMSE of this model on validation?

* 0.059
* 0.259
* 0.459
* 0.659

> Validation RMSE: 0.46

## Question 3

Now let's experiment with the `n_estimators` parameter

* Try different values of this parameter from 10 to 200 with step 10
* Set `random_state` to `1`
* Evaluate the model on the validation dataset

In [ ]:
def test_params(**params):
    """
    - Train a model with hyperparameter passed to function 
    - Predict  on validation set
    - Evaluate MSE
    """
    model = RandomForestRegressor(**params, random_state= 1).fit(train_dv, y_train)
    pred = model.predict(val_dv)
    val_rmse = mean_squared_error(y_val, pred,  squared=False)
    return val_rmse

def test_multiple_values(param_name, param_values):
    """
    For given param_name and range of values, train a model individually through function test_params
    and fetch-append validation RMSE
    """
    val_errors = []
    for value in param_values:
        params = {param_name: value}
        metric = test_params(**params)
        val_errors.append(round(metric, 5))
        print(f'Error evaluated for {param_name}:{value}')
    return val_errors

In [ ]:
# Compile param values and rmse values together
estimators_range = [i for i in range(10, 201, 10)]
rmse_list = list(zip(estimators_range,(test_multiple_values('n_estimators', estimators_range))))

In [ ]:
# rmse_list = [(10, 0.45985),
#  (20, 0.44783),
#  (30, 0.44512),
#  (40, 0.44323),
#  (50, 0.44223),
#  (60, 0.44153),
#  (70, 0.44087),
#  (80, 0.44076),
#  (90, 0.44024),
#  (100, 0.43978),
#  (110, 0.43933),
#  (120, 0.43914),
#  (130, 0.43926),
#  (140, 0.43911),
#  (150, 0.4391),
#  (160, 0.43891),
#  (170, 0.43887),
#  (180, 0.43905),
#  (190, 0.43895),
#  (200, 0.43894)]

In [ ]:
_, rmse = zip(*rmse_list)

In [ ]:
plot = plt.plot(estimators_range, rmse)
plt.xlabel('n_estimators')
plt.ylabel('RMSE')
plt.annotate('min RMSE point',(100, 0.193), xytext = (150, 0.1975), arrowprops=dict(arrowstyle="simple"\
                        ,connectionstyle="arc3,rad=0.3"))
plt.show()

After which value of `n_estimators` does RMSE stop improving?

- 10
- 50
- 70
- 120

> 120

## Question 4

Let's select the best `max_depth`:

* Try different values of `max_depth`: `[10, 15, 20, 25]`
* For each of these values, try different values of `n_estimators` from 10 till 200 (with step 10)
* Fix the random seed: `random_state=1`

In [ ]:
# scores = []

# max_depth_range = [10, 15, 20, 25]

# for depth in max_depth_range:
#     print(f'Evaluating for max depth: {depth}')
#     for num_estimators in estimators_range:
#         rf = RandomForestRegressor(n_estimators=num_estimators,
#                                     max_depth=depth,
#                                     random_state=1)
#         rf.fit(train_dv, y_train)
#         y_pred = rf.predict(val_dv)
#         rmse = mean_squared_error(y_val, y_pred, squared= True)
#         print('.',sep="")
#         scores.append((depth, num_estimators, rmse))
        

In [ ]:
columns = ['max_depth', 'n_estimators', 'rmse']
df_scores = pd.DataFrame(scores, columns=columns)

In [ ]:
for d in max_depth_range:
    df_subset = df_scores[df_scores.max_depth == d]
    
    plt.plot(df_subset.n_estimators, df_subset.rmse,
             label=f'max_depth={d}')
    plt.xlabel('n_estimators')
    plt.ylabel('RMSE')

plt.legend()

What's the best `max_depth`:

* 10
* 15
* 20
* 25

>15

Bonus question (not graded):

Will the answer be different if we change the seed for the model?

## Question 5

We can extract feature importance information from tree-based models. 

At each step of the decision tree learning algorith, it finds the best split. 
When doint it, we can calculate "gain" - the reduction in impurity before and after the split. 
This gain is quite useful in understanding what are the imporatant features 
for tree-based models.

In Scikit-Learn, tree-based models contain this information in the `feature_importances_` field. 

For this homework question, we'll find the most important feature:

* Train the model with these parametes:
    * `n_estimators=10`,
    * `max_depth=20`,
    * `random_state=1`,
    * `n_jobs=-1` (optional)
* Get the feature importance information from this model

In [ ]:
forest5 = RandomForestRegressor(n_estimators=10, max_depth= 20, random_state= 1, n_jobs= -1)



In [ ]:
forest5.fit(train_dv, y_train)

In [ ]:
imp_feature_q5 = np.argmax(forest5.feature_importances_)

dv.get_feature_names()[imp_feature_q5]

What's the most important feature? 

* `neighbourhood_group=Manhattan`
* `room_type=Entire home/apt`	
* `longitude`
* `latitude`

>room_type=Entire home/apt

## Question 6

Now let's train an XGBoost model! For this question, we'll tune the `eta` parameter

* Install XGBoost
* Create DMatrix for train and validation
* Create a watchlist
* Train a model with these parameters for 100 rounds:

```
xgb_params = {
    'eta': 0.3, 
    'max_depth': 6,
    'min_child_weight': 1,
    
    'objective': 'reg:squarederror',
    'nthread': 8,
    
    'seed': 1,
    'verbosity': 1,
}
```

In [ ]:
# !pip install xgboost

In [ ]:
import xgboost as xgb

In [ ]:
features = dv.get_feature_names()
train_dmat = xgb.DMatrix(train_dv, y_train, feature_names= features)
val_dmat = xgb.DMatrix(val_dv, y_val, feature_names= features)

In [ ]:
xgb_params = {
    'eta': 0.01, 
    'max_depth': 6,
    'min_child_weight': 1,

    'objective': 'reg:squarederror',
    'nthread': 8,

    'seed': 1,
    'verbosity': 1,
}

In [ ]:
watchlist = [(train_dmat, 'train'), (val_dmat, 'validation')]

In [ ]:
%%capture output
model = xgb.train(xgb_params, train_dmat, num_boost_round= 100,verbose_eval=5, evals= watchlist)

In [ ]:
def parse_xgb_output(output):
    results = []

    for line in output.stdout.strip().split('\n'):
        it_line, train_line, val_line = line.split('\t')

        it = int(it_line.strip('[]'))
        train = float(train_line.split(':')[1])
        val = float(val_line.split(':')[1])

        results.append((it, train, val))
    
    columns = ['num_iter', 'train_rmse', 'val_rmse']
    df_results = pd.DataFrame(results, columns=columns)
    return df_results

In [ ]:
eta03 = parse_xgb_output(output)


In [ ]:
eta03

In [ ]:
eta01 = parse_xgb_output(output)


In [ ]:
eta01

In [ ]:
eta001 = parse_xgb_output(output)


In [ ]:
eta001

Now change `eta` first to `0.1` and then to `0.01`

In [ ]:
plt.plot(eta03['num_iter'], eta03['train_rmse'], color = 'black', label = 'eta 0.3')
plt.plot(eta01['num_iter'], eta01['train_rmse'], color = 'blue', label = 'eta 0.1')
plt.plot(eta001['num_iter'], eta001['train_rmse'], color = 'red', label = 'eta 0.01')

plt.xlabel('Iteration')
plt.ylabel('RMSE')

plt.legend()

plt.show()

What's the best eta?

* 0.3
* 0.1
* 0.01

> 0.3

## Submit the results


Submit your results here: https://forms.gle/wQgFkYE6CtdDed4w8

It's possible that your answers won't match exactly. If it's the case, select the closest one.


## Deadline


The deadline for submitting is 20 October 2021, 17:00 CET (Wednesday). After that, the form will be closed.

